In [109]:
import pandas as pd
import numpy as np


import matplotlib
from matplotlib import pyplot as plt


#states = PATH1, PATH2, PATH3, PATH4 = 0, 1, 2, 3

V = np.array([3, 3, 2, 1, 2, 2, 1, 2, 3, 0])

# Transition Probabilities
A = np.array(((0.3, 0.2, 0.2, 0.3), (0.2, 0.3, 0.2, 0.3), (0.2, 0.2, 0.3, 0.3), (0.2, 0.2, 0.2, 0.4)))
A = A / np.sum(A)

# Emission Probabilities
B = np.array(((0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.1, 0.1, 0.1, 0.7)))
B = B / np.sum(B).reshape((-1, 1))

# Probabilities for the initial distribution
pi = np.array((0.2, 0.2, 0.2, 0.4))

def forward(V, A, B, pi):
    global alpha
    alpha = np.zeros((V.shape[0], A.shape[0]))
    alpha[0, :] = pi * B[:, V[0]]

    for t in range(1, V.shape[0]):
        for j in range(A.shape[0]):

            alpha[t, j] = alpha[t - 1].dot(A[:, j]) * B[j, V[t]]

    return alpha


def backward(V, A, B):
    global beta
    beta = np.zeros((V.shape[0], A.shape[0]))

    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((A.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(A.shape[0]):
            beta[t, j] = (beta[t + 1] * B[:, V[t + 1]]).dot(A[j, :])

    return beta


def baum_welch(V, A, B, pi, n_iter=100):
    global gamma, numerator, denominator, xi
    M = A.shape[0]
    T = len(V)

    for n in range(n_iter):
        alpha = forward(V, A, B, pi)
        beta = backward(V, A, B)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, A) * B[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * A[i, :] * B[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        A = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = B.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            B[:, l] = np.sum(gamma[:, V == l], axis=1)

        B = np.divide(B, denominator.reshape((-1, 1)))

    return {"A_hat":A, "B_hat":B, "Pi_hat": gamma[:,0]}

baum_welch(V, A, B, pi, n_iter=1)

{'A_hat': array([[0.33137496, 0.22091664, 0.22091664, 0.22679177],
        [0.22091664, 0.33137496, 0.22091664, 0.22679177],
        [0.22091664, 0.22091664, 0.33137496, 0.22679177],
        [0.20838237, 0.20838237, 0.20838237, 0.37485288]]),
 'B_hat': array([[0.12546498, 0.26411403, 0.51887847, 0.09154252],
        [0.12546498, 0.26411403, 0.51887847, 0.09154252],
        [0.12546498, 0.26411403, 0.51887847, 0.09154252],
        [0.04975425, 0.07349461, 0.1654372 , 0.71131393]]),
 'Pi_hat': array([0.05029605, 0.05029605, 0.05029605, 0.84911184])}

In [115]:
gamma.T

array([[0.05029605, 0.05029605, 0.05029605, 0.84911184],
       [0.06731086, 0.06731086, 0.06731086, 0.79806743],
       [0.28074427, 0.28074427, 0.28074427, 0.1577672 ],
       [0.29210906, 0.29210906, 0.29210906, 0.12367281],
       [0.29271264, 0.29271264, 0.29271264, 0.12186209],
       [0.29271515, 0.29271515, 0.29271515, 0.12185454],
       [0.29215893, 0.29215893, 0.29215893, 0.1235232 ],
       [0.28168094, 0.28168094, 0.28168094, 0.15495717],
       [0.08490169, 0.08490169, 0.08490169, 0.74529494],
       [0.27755123, 0.27755123, 0.27755123, 0.1673463 ]])

In [114]:
gamma[:,9]

array([0.27755123, 0.27755123, 0.27755123, 0.1673463 ])